- 了解数据如何收集，数据的格式
- 了解如何用深度学习训练
- 了解深度强化学习中的离线学习（在数据集中学习）是如何进行的
- 了解深度强化学习中的在线学习（直接在游戏中学习）是如何进行的

- 至于理论部分，有空再补充

# 1 collect_data.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/collect_data.py)

### 1.1 get_output()

对按键信息进行独热编码  
**注意，这部分的定义非常重要，后面有部分重要的代码需要根据这里的定义来编写**

更新：记录位移信息，允许使用道具（指伤药葫芦等道具）  
但是，这些信息暂时不参与训练，以后再用。位移和道具目前被统一归到其他类。  
参与训练的仍然是 攻击、防御、垫步、跳跃、其他类。

~~~python
def get_output(keys):    # 对按键信息进行独热编码

    output = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # 攻击、防御、垫步、跳跃和使用道具不能同时进行（指0.1秒内），但是可以和移动同时进行
    if   'J' in keys:
        output[0] = 1    # 等同于[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    elif 'K' in keys:
        output[1] = 1    # 等同于[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    elif 'LSHIFT' in keys:
        output[2] = 1    # 等同于[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    elif 'SPACE' in keys:
        output[3] = 1    # 等同于[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    elif 'R' in keys:
        output[5] = 1    # 等同于[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    else:
        output[4] = 1    # 等同于[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

    # 不能同时前后移动
    if   'W' in keys:
        output[6] = 1    # 等同于[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
    elif 'S' in keys:
        output[7] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

    # 不能同时左右移动
    if   'A' in keys:
        output[8] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    elif 'D' in keys:
        output[9] = 1    # 等同于[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

    return output
~~~

### 1.2 class Data_collection

~~~python
def __init__(self, target):
    self.target = target    # 目标
    self.dataset = list()    # 保存数据的容器
    self.save_path = os.path.join('The_battle_memory', self.target)    # 保存的位置
    if not os.path.exists(self.save_path):    # 确保保存的位置存在
        os.mkdir(self.save_path)

    self.step = 0    # 计步器
~~~

##### 1.2.1 保存数据方法

保存数据，统一名称为 f'training_data-{n}.npy'，例如 training_data-123.npy  
从1开始检测文件名是否存在于保存路径，直到检测到不存在就保存，主要是为了防止文件名重复不小心把原本的文件覆盖了

~~~python
def save_data(self):
    n = 1
    while True:    # 直到找到保存位置并保存就 break
        filename = f'training_data-{n}.npy'
        save_path = os.path.join(self.save_path, filename)
        if not os.path.exists(save_path):    # 没有重复的文件名就执行保存并退出
            print(save_path)
            np.save(save_path, self.dataset)
            break
        n += 1
~~~

##### 1.2.2 收集数据方法

按 'T' 开始，然后等待按 'P' 结束
~~~python
def collect_data(self):

    paused = True
    while True:
        last_time = time.time()
        keys = key_check()
        if paused:
            if 'T' in keys:
                paused = False
        else:

            self.step += 1

            """
            核心部分，看下面
            """

            if 'P' in keys:    # 结束，保存数据
                self.save_data()    # 保存数据
                break
~~~

**核心部分**  
- 记录数据的条件是非黑屏  
- 数据采集降频的目的是缓解数据不平衡和提高 action 与 reward 的对应程度
    - 按键检测的原理是检测当前键盘上正在按着的键，我们的操作有按键和松键，按键对应了多种动作，松键只对应1个动作（其他），所以会有大量数据集中在松键状态，而采集频率的增加则会加剧这种情况。
    - 游戏接受玩家的按键信号，然后让人物执行相应的动作，过一会玩家从屏幕观测到状态变化（生命值和架势的变化）。如果采集频率过高，会导致状态变化可能存在于好几个新状态之后， action 与 reward 不对应会导致训练出现严重的后果甚至训练结果是无效的。

~~~python
screen = get_screen()    # 获取屏幕图像
if not (np.sum(screen == 0) > 97200):    # 270 * 480 * 3 / 4 = 97200 ，当图像有1/4变成黑色（像素值为0）的时候停止暂停收集数据
    action_onehot = get_output(keys)    # 获取按键输出
    self.dataset.append([screen, action_onehot])    # 图像和输出打包在一起，保证一一对应

# 降低数据采集的频率，两次采集的时间间隔为0.1秒
t = 0.1-(time.time()-last_time)
if t > 0:
    time.sleep(t)
~~~

### 1.3 数据预览

**注：以下展示的是我本地的数据集，我没有上传，所以要运行以下代码，请先自己收集至少一个数据集**

- Data_quality (存放数据集对应的reward曲线)
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.png （第一个战斗数据的reward曲线）
-
- The_battle_memory
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.npy （第一个战斗数据）

In [ ]:
import os

import cv2
import numpy as np
import pandas as pd

target = 'Genichiro_Ashina' # 苇名弦一郎
# target = 'Inner_Genichiro' # 心中的弦一郎
# target = 'Isshin,_the_Sword_Saint' # 剑圣一心
# target = 'Inner_Isshin' # 心中的一心
path = os.path.join('The_battle_memory', target, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)

print(f'当前数据集所包含数据量：{data.shape[0]}, 每个数据都有 {data.shape[1]} 个数据，分别是图像数据和按键数据')

In [ ]:
screen = data[0][0]    # 选取了第一个数据的图像部分
print('图像', screen)
print('形状', screen.shape)

In [ ]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)
print('目前的训练只使用前五个  ', action_value[:5])

In [ ]:
# 统计动作分布
def motionCounts(Y):

    columns=['攻击', '弹反', '垫步', '跳跃', '无键', '道具', '向前', '向后', '向左', '向右']
    df = pd.DataFrame(Y, columns=columns, dtype=np.uint8)
    
    total = len(df)

    for motion in columns:
        motion_count = len(df)-df[motion].value_counts()[0]
        print(f'motion: {motion:>3}, 记录到的次数: {motion_count:>5}')

Y = np.array([i[1] for i in data])
motionCounts(Y)

In [ ]:
from pysekiro.get_vertices import roi

print()
print('以视频的形式展示数据。', '要你自己运行才看得见视频')
Remaining = len(data)
for screen, action_value in data:
    action = np.argmax(action_value)

    if   action == 0:
        a = '攻击'
    elif action == 1:
        a = '弹反'
    elif action == 2:
        a = '垫步'
    elif action == 3:
        a = '跳跃'
    elif action == 4:
        a = '其他'

    cv2.imshow('screen', screen)
    cv2.imshow('roi', roi(screen, x=140, x_w=340, y=30, y_h=230))
    cv2.waitKey(1)

    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{a}{action_value}', end='') # end='\n'
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

# 2 model.py

注意：我提供的模型不是最优的（*瞎搞的*），有能力的可以自己重写  
要求：输入形状相关参数（width, height, frame_count），输出相关参数（outputs），需要编译（model.compile）

# 3 train_with_dl.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/train_with_dl.py)

- **target** 指定训练的对象
- 注：必须 start < end
    - **start** 指定第一个训练的数据集
    - **end** 指定最后一个训练的数据集
- **batch_size** 训练时的批大小
- **epochs** 每个数据集训练的次数
    - 如果你搜集的数据足够多，并且是按照上面的要求做的，那完全可以设置成1
    - 因为既然每个数据都是一场完整的战斗，那么真正的epochs其实是数据集的数量，除非你每场战斗是风格都不一样
- **model_weights** 用以支持增量学习。默认为None，从零开始训练。
    - 当你搜集新的数据集之后想继续训练，可以把这个参数设置为要继续训练的模型的路径，这样就可以继续训练了
    - 当你在训练过程中发生了某种意外终止了训练，可以把这个参数设置为要继续训练的模型的路径，然后把发生意外时正在训练的那个数据集的序号设置为start，就可以继续训练了

~~~python
def train(
    target,
    start=1,
    end=1,
    batch_size=128,
    epochs=1,
    model_weights=None
    ):
    
    model = MODEL(ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT,
        outputs = n_action,
        model_weights = model_weights
    )
    model.summary()
    
    """
    核心部分，看下面
    """
~~~

**核心部分**
~~~python
model_weights = 'dl_weights.h5'

# 读取一个数据集训练，然后再读取下一个数据集训练，以此类推
for i in range(start, end+1):

    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)

    if os.path.exists(data_path):    # 确保数据集存在
        
        # 加载数据集
        data = np.load(data_path, allow_pickle=True)
        
        """
        训练部分，看下面
        """
        
    else:
        print(f'{filename} does not exist ')
~~~

**训练部分**
~~~python
# 数据集处理成预训练格式
X = np.array([cv2.resize(roi(i[0], x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)) for i in data]).reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
Y = np.array([i[1][:5] for i in data])

# 训练模型，然后保存
model.fit(X, Y, batch_size=batch_size, epochs=epochs, verbose=1)
model.save_weights(model_weights)
~~~

# 4 learn_offline.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_offline.py)

由 train_with_dl.py 升级而来

- **target** 指定训练的对象
- 注：必须 start < end
    - **start** 指定第一个训练的数据集
    - **end** 指定最后一个训练的数据集
- **model_weights** 用以支持增量学习。默认为None。没有指定需要增量学习的模型的路径的话，就从零开始训练。
    - 当你搜集新的数据集之后想继续训练，可以把这个参数设置为要继续训练的模型的路径，这样就可以继续训练了
    - 当你在训练过程中发生了某种意外终止了训练，可以把这个参数设置为要继续训练的模型的路径，然后把发生意外时正在训练的那个数据集的序号设置为start，就可以继续训练了
- **save_path** 指定模型权重保存的路径

~~~python
def learn_offline(
    target,
    start=1,
    end=1,
    model_weights=None,
    save_path=None
    )

    sekiro_agent = Sekiro_Agent(
        model_weights = model_weights,
        save_path = save_path
    )
~~~

~~~python
# 依次读取训练集进行离线学习
for i in range(start, end+1):

    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)

    if os.path.exists(data_path):    # 确保数据集存在

        # 加载数据集
        data = np.load(data_path, allow_pickle=True)

        """
        学习过程，看下面
        """
        
        sekiro_agent.save_evaluate_network()    # 这个数据学习完毕，保存网络权重
        sekiro_agent.reward_system.save_reward_curve(
            save_path = os.path.join('Data_quality', target, filename[:-4]+'.png')
        )    # 绘制 reward 曲线并保存

    else:
        print(f'{filename} does not exist ')
~~~

**学习过程**  
主要思路：获取 (状态S、动作A、奖励R 和 新状态S')，存储以供经验回放，然后目标网络和评估网络根据各自设置的更新频率更新自身网络参数  
~~~python
for step in range(len(data)-1):

    # ---------- (S, A, R, S') ----------

    screen = data[step][0]               # 状态S
    action = np.argmax(data[step][1])    # 动作A
    reward = sekiro_agent.reward_system.get_reward(get_status(screen))    # 奖励R
    next_screen = data[step+1][0]        # 新状态S'

    # ---------- store ----------

    # 集齐 (S, A, R, S')，开始存储
    sekiro_agent.replayer.store(
        cv2.resize(roi(screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)),    # 截取感兴趣区域并图像缩放
        action,
        reward,
        cv2.resize(roi(next_screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT))    # 截取感兴趣区域并图像缩放
    )    # 存储经验

    # ---------- learn ----------

    sekiro_agent.step = step
    sekiro_agent.learn()
~~~

# 5 learn_online.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_online.py)

由 collect_data.py 变化而来

- **model_weights** 指定模型权重的路径。
- **save_path** 指定模型权重保存的路径。注：设置该参数的同时会开始训练模式

~~~python
def learn_online(model_weights=None, save_path=None):
    
    sekiro_agent = Sekiro_Agent(
        model_weights=model_weights,
        save_path = save_path
    )
    
    if save_path:
        train = True
    else:
        train = False
~~~

按 'T' 开始，然后等待按 'P' 结束
~~~python
paused = True

step = 0    # 计步器

while True:

    last_time = time.time()
    keys = key_check()
    if paused:
        screen = get_screen()    # 首个 状态S，但是在按 'T' 之前，它会不断更新
        if 'T' in keys:
            paused = False
    else:

        step += 1
        
        """
        核心部分，看下面
        """

        # 降低数据采集的频率，两次采集的时间间隔为0.1秒
        t = 0.1-(time.time()-last_time)
        if t > 0:
            time.sleep(t)

        if 'P' in keys:
            if train:
                sekiro_agent.save_evaluate_network()    # 学习完毕，保存网络权重
            sekiro_agent.reward_system.save_reward_curve(save_path='learn_online.png')    # 绘制 reward 曲线并保存在当前目录
            break
~~~

**核心部分**  
- 按 'T' 之后，获得首个状态S，然后依次获得 动作A、奖励R 和 新状态S'，再进入第二个轮回  
- 第二个轮回开始
    1. 原本的 **新状态S'** 变成 **当前状态S**
    2. 由 **当前状态S** 选取 **动作A**
    3. 由 **当前状态S** 获取 **奖励R**
    4. 观测 **新状态S'**
    - 进入下一个轮回
- 这样做的目的是
    1. 集齐 (S, A, R, S')
    2. 保证 状态S 和 新状态S' 连续

~~~python
# ---------- (S, A, R, S') ----------

action = sekiro_agent.choose_action(screen, train)    # 动作A
reward = sekiro_agent.reward_system.get_reward(get_status(screen, show=True))    # 奖励R
next_screen = get_screen()    # 新状态S'

# ---------- 下一个轮回 ----------

screen = next_screen    # 状态S

"""
训练部分，看下面
"""
~~~

**训练部分**  
~~~python
if train:
    if not (np.sum(screen == 0) > 97200):    # 270 * 480 * 3 / 4 = 97200 ，当图像有1/4变成黑色（像素值为0）的时候停止暂停存储数据

        # ---------- store ----------

        # 集齐 (S, A, R, S')，开始存储
        sekiro_agent.replayer.store(
            cv2.resize(roi(screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT)),    # 截取感兴趣区域并图像缩放
            action,
            reward,
            cv2.resize(roi(next_screen, x, x_w, y, y_h), (ROI_WIDTH, ROI_HEIGHT))    # 截取感兴趣区域并图像缩放
        )    # 存储经验

        # ---------- learn ----------

        sekiro_agent.step += 1
        sekiro_agent.learn()
~~~

# 6 Agent.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/Agent.py)

[参考来源1](https://github.com/ZhiqingXiao/rl-book/blob/master/chapter10_atari/BreakoutDeterministic-v4_tf.ipynb)  
[参考来源2](https://mofanpy.com/tutorials/machine-learning/reinforcement-learning/DQN3/)  

## 6.1 class RewardSystem

计算 reward 的原理
~~~python
# 约束状态值的上下限，防止异常值和特殊值的影响。
def limit(value, lm1, lm2):

    if value < lm1:
        return lm1
    elif value > lm2:
        return lm2
    else:
        return value

# 每个状态的计算方法：(现在的状态 - 过去的状态) * 正负强化权重，然后约束上下限
s1 = limit((self.status[0] - self.past_status[0]) *  1, -152, +76)    # 自身生命
s2 = limit((self.status[1] - self.past_status[1]) * -1, -10, +10)    # 自身架势
t1 = limit((self.status[2] - self.past_status[2]) * -1, -20,   0)    # 目标生命
t2 = limit((self.status[3] - self.past_status[3]) *  1, -10, +10)    # 目标架势

reward = 0.2 * (s1 + t1) + 0.8 * (s2 + t2)
~~~

## 6.2 class DQNReplayer

这个不是我写的的  
[来源](https://github.com/ZhiqingXiao/rl-book/blob/master/chapter10_atari/BreakoutDeterministic-v4_tf.ipynb)

## 6.3 class Sekiro_Agent

~~~python
def __init__(
    self,
    n_action = n_action, 
    gamma = 0.99,
    batch_size = 128,
    replay_memory_size = 20000,
    epsilon = 1.0,
    epsilon_decrease_rate = 0.999,
    update_freq = 100,
    target_network_update_freq = 300,
    model_weights = None,
    save_path = None
):
    self.n_action = n_action    # 动作数量

    self.gamma = gamma    # 奖励衰减

    self.batch_size = batch_size                    # 样本抽取数量
    self.replay_memory_size = replay_memory_size    # 记忆容量

    self.epsilon = epsilon                                # 探索参数
    self.epsilon_decrease_rate = epsilon_decrease_rate    # 探索衰减率

    self.update_freq = update_freq    # 训练评估网络的频率
    self.target_network_update_freq = target_network_update_freq    # 更新目标网络的频率

    self.model_weights = model_weights    # 指定读取的模型参数的路径
    self.save_path = save_path            # 指定模型权重保存的路径
    if not self.save_path:
        self.save_path = 'tmp_weights.h5'

    self.evaluate_net = self.build_network()    # 评估网络
    self.target_net = self.build_network()      # 目标网络
    self.reward_system = RewardSystem()                     # 奖惩系统
    self.replayer = DQNReplayer(self.replay_memory_size)    # 经验回放

    self.step = 0    # 计步
~~~

### 6.3.1 评估网络和目标网络的构建方法
~~~python
def build_network(self):
    model = MODEL(ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT,
        outputs = self.n_action,
        model_weights = self.model_weights
    )
    return model
~~~

### 6.3.2 行为选择与执行方法
~~~python
def choose_action(self, screen, train):
    if train:
        r = np.random.rand()
    else:
        r = 1.01    # 永远大于 self.epsilon

    # train = True 开启探索模式
    if r < self.epsilon:
        self.epsilon *= self.epsilon_decrease_rate    # 逐渐减小探索参数, 降低行为的随机性
        action = np.random.randint(self.n_action)

    # train = False 直接进入这里
    else:
        screen = roi(screen, x, x_w, y, y_h)
        q_values = self.evaluate_net.predict([screen.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)])[0]
        action = np.argmax(q_values)

    # 执行动作
    act(action)

    return action
~~~

### 6.3.3 学习方法
~~~python
def learn(self):

    if self.step >= self.batch_size and self.step % self.update_freq == 0:    # 更新评估网络

        if self.step % self.target_network_update_freq == 0:    # 更新目标网络
            self.update_target_network() 

        # 经验回放
        screens, actions, rewards, next_screens = self.replayer.sample(self.batch_size)

        screens = screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
        next_screens = next_screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)

        # 计算回报的估计值
        q_next = self.target_net.predict(next_screens)
        q_target = self.evaluate_net.predict(screens)
        q_target[range(self.batch_size), actions] = rewards + self.gamma * q_next.max(axis=-1)

        self.evaluate_net.fit(screens, q_target, verbose=0)
~~~

### 6.3.4 更新目标网络权重方法
~~~python
def update_target_network(self):
    self.target_net.set_weights(self.evaluate_net.get_weights())
~~~

### 6.3.5 保存评估网络权重方法
~~~python
def save_evaluate_network(self):
    self.evaluate_net.save_weights(self.save_path)
~~~